Note: No imports in the notebook

# main.py

In [4]:
data_folder = './data'
dataset = load_imgs(data_folder)

model = autoencoder_conv()

train_data, test_data = train_test_split(dataset, test_size=0.33, random_state=1)

optimizations = opts()
train(model, train_data, *optimizations, num_epochs = 1)

# models.py

In [3]:
class autoencoder_conv(nn.Module):
    def __init__(self):
        super(autoencoder_conv, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def improve(self, train_data_iter, optimizer, loss_func):
        for imgs, _ in train_data_iter:
            input = imgs
            output = self(imgs)
            
            loss = loss_func(input, output)
            optimize(optimizer, loss)

In [5]:
class autoencoder_lin(nn.Module):
    def __init__(self):
        super(autoencoder_lin, self).__init__()
        self.encoder = nn.Sequential(
            View((-1, 28*28)),
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh(),
            View((-1, 1, 28, 28)))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def improve(self, train_data_iter, optimizer, loss_func):
        for imgs, _ in train_data_iter:
            input = imgs
            output = self(imgs)
            
            loss = loss_func(input, output)
            optimize(optimizer, loss)
            
class View(nn.Module):
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

# lib.py

In [2]:
def load_imgs(img_folder, iterator = True):
    img_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # dataset = ImageFolder('./data', trans..=...)
    dataset = MNIST(img_folder, download=True, transform=img_transform)
    
    
    return dataset


def opts():
    learning_rate = 1e-3
    optimizer = torch.optim.Adam(
        model.parameters(), lr=learning_rate, weight_decay=1e-5)
    loss_func = nn.MSELoss()
    return (optimizer, loss_func)


def optimize(optimizer, loss):
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

def train(model, data, optimizer, loss_func, num_epochs = 100):
    
    for epoch in range(num_epochs):
        data_iter = DataLoader(data, batch_size = 128)
        model.improve(data_iter, optimizer, loss_func)

# needed_libs.py

In [1]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from sklearn.model_selection import train_test_split

from torch.utils.tensorboard import SummaryWriter

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')